## <div style="padding: 20px;color:white;margin:10;font-size:90%;text-align:left;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://w0.peakpx.com/wallpaper/957/661/HD-wallpaper-white-marble-white-stone-texture-marble-stone-background-white-stone.jpg)"><b><span style='color:black'> Load remote Parquet file</span></b> </div>

In [1]:
#Import the necessary libraries
import duckdb 
import os 

In [2]:
#Load the httpfs to load data from a remote repo.  
conn = duckdb.connect('taxi_NYC.db')
# conn.execute("INSTALL httpfs;")
# conn.execute("LOAD httpfs;")

In [3]:
#Define the table data types before loading the data. 
conn.execute("""
    CREATE TABLE IF NOT EXISTS yellow_taxi_may (
        VendorID INTEGER,
        tpep_pickup_datetime TIMESTAMP,
        tpep_dropoff_datetime TIMESTAMP,
        passenger_count INTEGER,
        trip_distance FLOAT,
        RatecodeID INTEGER,
        store_and_fwd_flag STRING,
        PULocationID INTEGER,
        DOLocationID INTEGER,
        payment_type INTEGER,
        fare_amount FLOAT,
        extra FLOAT,
        mta_tax FLOAT,
        tip_amount FLOAT,
        tolls_amount FLOAT,
        improvement_surcharge FLOAT,
        total_amount FLOAT,
        congestion_surchage FLOAT,
        Airport_fee FLOAT
    );
    INSERT INTO yellow_taxi_may
    SELECT * FROM read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet');
""")


In [3]:
#View the top ten records. 
conn.execute("""SELECT * FROM yellow_taxi_may LIMIT 10;""").fetch_df()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surchage,Airport_fee
0,1,2024-05-01 00:59:15,2024-05-01 01:23:50,1,6.10,1,N,138,145,1,28.200001,7.75,0.5,5.00,0.00,1.0,42.450001,0.0,1.75
1,2,2024-04-30 23:58:26,2024-05-01 00:29:42,1,11.23,1,N,138,249,1,46.400002,6.00,0.5,8.72,0.00,1.0,66.870003,2.5,1.75
2,2,2024-05-01 00:57:17,2024-05-01 01:14:15,1,9.02,1,N,138,170,1,35.900002,6.00,0.5,10.57,6.94,1.0,65.160004,2.5,1.75
3,2,2024-05-01 00:24:47,2024-05-01 00:48:51,1,6.53,1,N,87,133,1,30.299999,1.00,0.5,7.06,0.00,1.0,42.360001,2.5,0.00
4,2,2024-05-01 00:11:20,2024-05-01 00:52:10,1,14.38,1,N,161,165,1,61.799999,1.00,0.5,0.00,0.00,1.0,66.800003,2.5,0.00
5,2,2024-05-01 00:54:41,2024-05-01 01:14:00,1,9.19,1,N,138,151,1,37.299999,6.00,0.5,10.70,6.94,1.0,64.190002,0.0,1.75
6,2,2024-05-01 00:06:44,2024-05-01 00:27:44,1,4.52,1,N,186,33,1,23.299999,1.00,0.5,5.09,0.00,1.0,33.389999,2.5,0.00
7,1,2024-05-01 00:11:37,2024-05-01 00:20:05,1,2.70,1,N,237,166,1,12.800000,3.50,0.5,3.20,0.00,1.0,21.000000,2.5,0.00
8,1,2024-05-01 00:37:26,2024-05-01 00:54:06,1,4.30,1,N,237,116,2,20.500000,3.50,0.5,0.00,0.00,1.0,25.500000,2.5,0.00
9,2,2024-05-01 00:05:24,2024-05-01 00:29:26,1,4.28,1,N,114,142,1,25.400000,1.00,0.5,6.08,0.00,1.0,36.480000,2.5,0.00


In [23]:
conn.execute("""SELECT CONCAT(PULocationID, '-', DOLocationID) AS route, tip_amount, fare_amount, passenger_count, payment_type, 
                        trip_distance, store_and_fwd_flag, extra
                FROM yellow_taxi_may
                ORDER BY tip_amount DESC
                LIMIT 10;""").fetch_df()

,route,tip_amount,fare_amount,passenger_count,payment_type,trip_distance,store_and_fwd_flag,extra
0,162-68,437.839996,19.799999,1,1,3.000000,N,2.5
1,45-88,430.899994,12.800000,0,1,1.500000,N,2.5
2,193-145,300.000000,5.800000,2,1,0.600000,N,0.0
3,265-265,281.000000,3.700000,1,1,35.200001,N,0.0
4,68-265,228.000000,26.799999,1,1,3.500000,N,5.0
5,125-114,200.000000,10.000000,2,1,1.220000,N,1.0
6,142-230,200.000000,11.400000,1,1,1.800000,N,3.5
7,237-144,200.000000,35.200001,1,1,5.650000,N,0.0
8,132-186,200.000000,70.000000,1,1,22.639999,N,0.0
9,264-264,180.000000,23.000000,1,1,0.000000,N,0.0


In [15]:
conn.execute(""" SELECT COUNT(DISTINCT CONCAT(PULocationID, '-', DOLocationID)) AS routes_count, tip_amount, fare_amount, passenger_count
                FROM yellow_taxi_may
                GROUP BY tip_amount, fare_amount, passenger_count
                ORDER BY tip_amount DESC
                LIMIT 10""").fetch_df()

,"count(DISTINCT concat(PULocationID, '-', DOLocationID))",tip_amount,fare_amount,passenger_count
0,1,437.839996,19.799999,1
1,1,430.899994,12.800000,0
2,1,300.000000,5.800000,2
3,1,281.000000,3.700000,1
4,1,228.000000,26.799999,1
5,1,200.000000,10.000000,2
6,1,200.000000,35.200001,1
7,1,200.000000,70.000000,1
8,1,200.000000,11.400000,1
9,1,180.000000,23.000000,1


In [16]:
conn.execute("""SELECT DISTINCT(PULocationID), SUM(passenger_count) AS total_passengers
                FROM yellow_taxi_may
                GROUP BY PULocationID
                ORDER BY total_passengers DESC
                LIMIT 10""").fetch_df()

,PULocationID,total_passengers
0,132,243007.0
1,237,220403.0
2,161,216962.0
3,236,197312.0
4,138,165665.0
5,230,165267.0
6,142,153121.0
7,162,151752.0
8,186,140954.0
9,163,133769.0


In [17]:
conn.execute("""SELECT DISTINCT CONCAT(PULocationID, '-', DOLocationID) AS route, SUM(passenger_count) AS total_passengers, VendorID
                FROM yellow_taxi_may
                GROUP BY route, VendorID
                ORDER BY total_passengers DESC
                LIMIT 10""").fetch_df()

,route,total_passengers,VendorID
0,237-236,26339.0,2
1,236-237,22226.0,2
2,237-237,17902.0,2
3,236-236,17421.0,2
4,132-230,13333.0,2
5,132-132,12715.0,2
6,161-237,12607.0,2
7,237-161,10605.0,2
8,161-236,10295.0,2
9,138-230,10034.0,2


In [21]:
conn.execute("""SELECT VendorID, SUM(passenger_count) AS total_pass, SUM(tip_amount) AS total_tips
                FROM yellow_taxi_may
                GROUP BY VendorID
                ORDER BY total_pass DESC
             """).fetch_df_chunk()

,VendorID,total_pass,total_tips
0,2,3436481.0,9.616123e+06
1,1,955812.0,2.754947e+06
2,6,NaN,0.000000e+00


In [18]:
conn.execute("""
    SELECT dayname(tpep_pickup_datetime) AS pickup_day, PULocationID, COUNT(passenger_count) AS pickup_count
    FROM yellow_taxi_may
    GROUP BY pickup_day, PULocationID
    ORDER BY pickup_count DESC
    
""").fetch_df()


,pickup_day,PULocationID,pickup_count
0,Wednesday,237,34250
1,Thursday,237,33615
2,Thursday,161,32854
3,Wednesday,161,32487
4,Thursday,236,29516
...,...,...,...
1724,Wednesday,57,0
1725,Tuesday,118,0
1726,Wednesday,59,0
1727,Wednesday,172,0


In [10]:
conn.execute("""
    SELECT 
        dayname(tpep_pickup_datetime) AS pickup_day, 
        COUNT(passenger_count) AS pickup_count,
        VendorID
    FROM 
        yellow_taxi_may
    GROUP BY 
        pickup_day, VendorID
    ORDER BY 
        pickup_count DESC
""").fetch_df()


,pickup_day,pickup_count,VendorID
0,Thursday,448069,2
1,Wednesday,434223,2
2,Friday,418778,2
3,Saturday,328424,2
4,Tuesday,326911,2
5,Sunday,281088,2
6,Monday,270256,2
7,Thursday,146327,1
8,Wednesday,143357,1
9,Friday,139270,1


In [13]:
conn.execute("""
    SELECT 
        dayname(tpep_pickup_datetime) AS pickup_day, 
        HOUR(tpep_pickup_datetime) AS pickup_hour,
        COUNT(passenger_count) AS pickup_count
    FROM 
        yellow_taxi_may
    GROUP BY 
        pickup_day, pickup_hour
    ORDER BY 
        pickup_day, pickup_hour, pickup_count DESC 
""").fetch_df()


,pickup_day,pickup_hour,pickup_count
0,Friday,0,16742
1,Friday,1,9192
2,Friday,2,5165
3,Friday,3,3228
4,Friday,4,2323
...,...,...,...
163,Wednesday,19,37620
164,Wednesday,20,36129
165,Wednesday,21,39528
166,Wednesday,22,32779


In [34]:
conn.execute("""
    SELECT 
        DISTINCT(DOLocationID), 
        VendorID, 
        store_and_fwd_flag, 
        COUNT(*) AS number, 
        dayname(tpep_pickup_datetime) AS pickup_day, 
        HOUR(tpep_pickup_datetime) AS pickup_hour
    FROM 
        yellow_taxi_may
    WHERE 
        store_and_fwd_flag = 'Y'
    GROUP BY 
        VendorID, store_and_fwd_flag, DOLocationID, pickup_day, pickup_hour
    ORDER BY 
        number DESC
""").fetch_df()


,DOLocationID,VendorID,store_and_fwd_flag,number,pickup_day,pickup_hour
0,236,2,Y,72,Monday,20
1,236,2,Y,66,Monday,16
2,141,2,Y,65,Monday,20
3,238,2,Y,64,Monday,21
4,236,2,Y,62,Monday,21
...,...,...,...,...,...,...
7590,255,1,Y,1,Thursday,14
7591,127,1,Y,1,Thursday,14
7592,151,1,Y,1,Friday,18
7593,79,1,Y,1,Friday,23


In [3]:
conn.close()